# 表情辨識實作流程（Ubuntu 開發環境）

In [13]:
# 安裝必要套件
# !pip install mediapipe opencv-python matplotlib gradio tensorflow keras

## Mediapipe Face Detection + Bounding Box Crop

In [14]:
import mediapipe as mp
import cv2
import numpy as np
import matplotlib.pyplot as plt

def detect_face(image_path):
    img = cv2.imread(image_path)
    mp_face = mp.solutions.face_detection
    face_detection = mp_face.FaceDetection(model_selection=0, min_detection_confidence=0.5)
    results = face_detection.process(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
    bbox = None
    if results.detections:
        for det in results.detections:
            bboxC = det.location_data.relative_bounding_box
            h, w, _ = img.shape
            bbox = int(bboxC.xmin * w), int(bboxC.ymin * h), \
                   int(bboxC.width * w), int(bboxC.height * h)
            face_crop = img[bbox[1]:bbox[1]+bbox[3], bbox[0]:bbox[0]+bbox[2]]
            return img, face_crop, bbox
    return img, None, None


## 載入 FER-2013 表情分類模型（Keras 範例）

In [15]:
from tensorflow.keras.models import load_model

# 模型讀取與預測
def predict_emotion(face_crop, model_path='./model/fer_model_best.h5'):
    model = load_model(model_path)
    input_face = cv2.cvtColor(face_crop, cv2.COLOR_BGR2GRAY)
    input_face = cv2.resize(input_face, (48, 48)) / 255.0
    input_face = input_face.reshape(1, 48, 48, 1)
    prediction = model.predict(input_face)
    return prediction


## 結果視覺化（OpenCV / matplotlib）

In [16]:
def visualize_prediction(img, bbox, prediction):
    emotion_labels = ['Angry', 'Disgust', 'Fear', 'Happy', 'Sad', 'Surprise', 'Neutral']
    pred_label = emotion_labels[np.argmax(prediction)]
    
    if bbox:
        cv2.rectangle(img, (bbox[0], bbox[1]),
                      (bbox[0]+bbox[2], bbox[1]+bbox[3]), (0, 255, 0), 2)
        cv2.putText(img, pred_label, (bbox[0], bbox[1] - 10),
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

    plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
    plt.axis('off')
    plt.show()

## Bonus：Gradio 圖形介面

In [ ]:
import gradio as gr

def gradio_interface(image):
    image_bgr = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    cv2.imwrite("temp.jpg", image_bgr)
    img, face_crop, bbox = detect_face("temp.jpg")
    if face_crop is not None:
        pred = predict_emotion(face_crop)
        visualize_prediction(img, bbox, pred)
    return cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

gr.Interface(fn=gradio_interface,
             inputs=gr.Image(type="numpy"),
             outputs=gr.Image(type="numpy"),
             title="Facial Expression Classifier").launch()

* Running on local URL:  http://127.0.0.1:7862

To create a public link, set `share=True` in `launch()`.


I0000 00:00:1744519041.590985  192318 gl_context_egl.cc:85] Successfully initialized EGL. Major : 1 Minor: 5
I0000 00:00:1744519041.608648  192340 gl_context.cc:369] GL version: 3.1 (OpenGL ES 3.1 Mesa 23.2.1-1ubuntu3.1~22.04.3), renderer: D3D12 (NVIDIA GeForce RTX 3070)
W0000 00:00:1744519041.620830  192333 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
/mnt/c/Users/patti/Desktop/Conestoga Courses/ConestogaCollegeWorkplace/venv_wsl/lib/python3.12/site-packages/keras/src/layers/convolutional/base_separable_conv.py:104: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(
Traceback (most recent call last):
  File "/mnt/c/Users/patti/Desktop/Conestoga Courses/ConestogaCollegeWorkplace/venv_wsl/lib/python3.12/site-packages/keras/src/ops/operatio